## Subpage Link Extractor

<br>

### Development Environment

In [ ]:
%pip install selenium 

In [49]:
import re
import json
import urllib
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Subpage Link Extractor

In [4]:
def getdata(url):
	r = requests.get(url)
	return r.text

In [44]:
def get_links(website_link, subpage_txt_file_name):

	dict_href_links = {}
	
	html_data = getdata(website_link)
	soup = BeautifulSoup(html_data, "html.parser")
	list_links = []
	list_page_links = []
	list_page_numbers = []
	list_page_strings = []

	for link in soup.find_all("a", href=True):

		if str(link["href"]).startswith((str(website_link))):
			list_links.append(link["href"])

		if str(link["href"]).startswith("/"):
			if link["href"] not in dict_href_links:
				dict_href_links[link["href"]] = None
				link_with_www = website_link + link["href"][1:]
				list_links.append(link_with_www)

				#nested subpage

				html_data = getdata(link_with_www)
				soup = BeautifulSoup(html_data, "html.parser")

				for link in soup.find_all("a", href=True):

					if str(link["href"]).startswith("/"):
						if link["href"] not in dict_href_links:
							dict_href_links[link["href"]] = None
							
							link_with_www = website_link + link["href"][1:]
							list_links.append(link_with_www)

	#nested subpage "page="

	for link in list_links:
		if "page=" in link:
			number = int(re.sub(r'[^0-9]', '', link))
			page_string = re.sub(r"[0-9]", "", link)
			list_page_links.append(link)
			list_page_numbers.append(number)

	max_page_numbers = max(list_page_numbers)

	for page_number in range(max_page_numbers):
		page = page_string + str(page_number + 1)


		html_data = getdata(page)
		soup = BeautifulSoup(html_data, "html.parser")
		bs4_element_script = soup.find_all("script", type='application/ld+json')
		bs4_element_json = [json.loads(x.string) for x in bs4_element_script]

		for bs4_element_dict in bs4_element_json:
			item_list_element = bs4_element_dict['itemListElement']
			for i in range(len(item_list_element)):
				link = item_list_element[i]['item']['@id']
				list_links.append(link)

	dict_links = dict.fromkeys(list_links)
	list_links = list(dict_links)
	list_links = sorted(list_links)
	
	with open(subpage_txt_file_name, 'a', encoding='utf-8') as fp:
		fp.write("\n".join(list_links))

	return list_links

In [55]:
url = "https://odanttoi.tistory.com/"
subpage_txt_file_name = "web_subpage_link.txt"
list_links = get_links(url, subpage_txt_file_name)
subpage_df = pd.DataFrame({"subpage": list_links})

In [57]:
pd.set_option('display.max_colwidth', None)
subpage_df 

,subpage
0,https://odanttoi.tistory.com/
1,https://odanttoi.tistory.com/10
2,https://odanttoi.tistory.com/100
3,https://odanttoi.tistory.com/101
4,https://odanttoi.tistory.com/102
...,...
162,https://odanttoi.tistory.com/manage
163,https://odanttoi.tistory.com/page/about
164,https://odanttoi.tistory.com/page/country
165,https://odanttoi.tistory.com/tag/%ED%86%B5%EA%B3%84%ED%95%99


### Reference

<b>Plain English</b>
<br>[Scraping all Subpages of a Website in Python](https://plainenglish.io/blog/scraping-the-subpages-on-a-website-ea2d4e3db113)
<br><br><b>Stackoverflow</b>
<br>[Parsing application ld+Json with Beautifulsoup (findAll)](https://stackoverflow.com/questions/68460844/parsing-application-ldjson-with-beautifulsoup-findall)
